### This code removes drogued drifters by temporally selecting drifter data after the date they lost their drogue. Next, it separates ultimately beaching drifters from never-beaching drifters and saves them to csv files

In [1]:
# set working directory
import os
os.chdir('/dat1/openonic/Drifters') # directory

In [2]:
# dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

/usr/local/anaconda3/lib/python3.7/site-packages/xarray/backends/cfgrib_.py:28: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  "Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. "


In [3]:
# load file
gdp = xr.open_dataset('gdp.nc', decode_times=False)

/usr/local/anaconda3/lib/python3.7/site-packages/xarray/backends/plugins.py:65: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [4]:
gdp

<xarray.Dataset>
Dimensions:                (traj: 17324, obs: 165754333)
Coordinates:
    ID                     (traj) int64 ...
    longitude              (obs) float32 ...
    latitude               (obs) float32 ...
    time                   (obs) int64 ...
    ids                    (obs) int64 ...
Dimensions without coordinates: traj, obs
Data variables: (12/53)
    rowsize                (traj) int64 ...
    location_type          (traj) bool ...
    WMO                    (traj) int32 ...
    expno                  (traj) int32 ...
    deploy_date            (traj) float64 ...
    deploy_lon             (traj) float32 ...
    ...                     ...
    err_sst                (obs) float32 ...
    err_sst1               (obs) float32 ...
    err_sst2               (obs) float32 ...
    flg_sst                (obs) int8 ...
    flg_sst1               (obs) int8 ...
    flg_sst2               (obs) int8 ...
Attributes: (12/15)
    title:             Global Drifter Program hourly drifting buoy collection
    history:           Version 2.00.  Metadata from dirall.dat and deplog.dat
    Conventions:       CF-1.6
    date_created:      2022-01-14T11:16:01.998226
    publisher_name:    GDP Drifter DAC
    publisher_email:   aoml.dftr@noaa.gov
    ...                ...
    metadata_link:     https://www.aoml.noaa.gov/phod/dac/dirall.html
    contributor_name:  NOAA Global Drifter Program
    contributor_role:  Data Acquisition Center
    institution:       NOAA Atlantic Oceanographic and Meteorological Laboratory
    acknowledgement:   Elipot et al. (2022) to be submitted. Elipot et al. (2...
    summary:           Global Drifter Program hourly data

In [5]:
import time
# set up returned dataframe
variables = ['id', 'time', 'lat', 'lon', 've', 'vn'] 
data_list = []

# extract values
ids = gdp.ids.values
times = gdp.time.values
lats = gdp.latitude.values
lons = gdp.longitude.values
ves = gdp.ve.values
vns = gdp.vn.values

In [6]:
# extract values and set to pandas dataframe
start_time = time.time()
data_list = [
    {'id': i, 'time': time, 'lat': lat, 'lon': lon, 've': ve, 'vn': vn}
    for i, time, lat, lon, ve, vn in zip(ids, times, lats, lons, ves, vns)
]

full_data = pd.DataFrame(data_list, columns=variables)

end_time = time.time()

elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 598.0741069316864 seconds


In [7]:
beach_ids = []
unbeach_ids = []

for i in range(len(gdp.type_death)):
    if gdp.type_death[i].item() == 1:
        beach_ids.append(gdp.ID[i].item())
    else:
        unbeach_ids.append(gdp.ID[i].item())

In [8]:
beach_full = full_data[full_data['id'].isin(beach_ids)] # get all ids that beached
unbeach_full = full_data[full_data['id'].isin(unbeach_ids)] # get all ids that didn't beach

In [9]:
min_time = gdp.drogue_lost_date.values # get time of drogue lost
min_ID = gdp.drogue_lost_date.coords['ID'].values # get ids of drogue lost
min_time_df = pd.DataFrame({'min_time': min_time, 'id': min_ID}) # put them in a DF together
beach_min_time = min_time_df[min_time_df['id'].isin(beach_ids)] # separate into beached
unbeach_min_time = min_time_df[min_time_df['id'].isin(unbeach_ids)] # and unbeached

In [10]:
# beached
merged_beach = pd.merge(beach_full, beach_min_time, left_on='id', right_on='id', how='inner')

# remove drogued values using time of drogue loss
undrogued_beach_ = merged_beach[merged_beach['time'] >= merged_beach['min_time']]

# remove drogue loss time from df
undrogued_beach = undrogued_beach_.drop(columns=['min_time'])

In [11]:
# unbeached
merged_unbeach = pd.merge(unbeach_full, unbeach_min_time, left_on='id', right_on='id', how='inner')

# remove drogued values using time of drogue loss
undrogued_unbeach_ = merged_unbeach[merged_unbeach['time'] >= merged_unbeach['min_time']]

# remove drogue loss time from df
undrogued_unbeach = undrogued_unbeach_.drop(columns=['min_time'])

In [12]:
# add column to undrogued_beach 'time to beach'
# get the last time values of beached
last_points = undrogued_beach.drop_duplicates(subset='id', keep='last')

def find_time_to_beach(beach, beach_last,time_between_register_beaching_and_actually_beaching):
    # DF to Numpy Arrays
    beach_array = beach.to_numpy()
    beach_last_array = beach_last.to_numpy()

    # Create a dictionary to store the last known time for each ID
    last_time_dict = dict(zip(beach_last_array[:, 0], beach_last_array[:, 1]))

    beach_time = []

    # Iterate through beach array and calculate time differences
    for row in beach_array:
        current_ID = row[0]
        last_time = last_time_dict.get(current_ID, None)
        
        if last_time is not None:
            current_time = row[1]
            if current_time != last_time: # prevent 0 value --> infinite time as beach_last exists within beach
                time_difference = last_time - current_time
                beach_time.append(time_difference)
            if current_time == last_time: # keep lat, lon, and values for hist shape the same
                beach_time.append(time_between_register_beaching_and_actually_beaching)

    return beach_time

time_between_register_beaching_and_actually_beaching = 0 # immediate beaching (?)

beach_time = find_time_to_beach(undrogued_beach, last_points, time_between_register_beaching_and_actually_beaching)

In [13]:
undrogued_beach['time_to_beach'] = beach_time

In [14]:
#undrogued_unbeach.to_csv('undrogued_unbeach.csv', index=False) # save to csv

In [15]:
#undrogued_beach.to_csv('undrogued_beach.csv', index=False) # save to csv

In [16]:
# drogued

In [17]:
min_time = gdp.drogue_lost_date.values # get time of drogue lost
min_ID = gdp.drogue_lost_date.coords['ID'].values # get ids of drogue lost
min_time_df = pd.DataFrame({'min_time': min_time, 'id': min_ID}) # put them in a DF together

merged = pd.merge(full_data['id'], min_time_df, left_on='id', right_on='id', how='inner')

# remove drogued values using time of drogue loss
drogued = full_data[full_data['time'] <= merged['min_time']]

In [18]:
#drogued.to_csv('drogued.csv', index=False) # save to csv

In [19]:
# separate drogued into beach and unbeach

In [20]:
# beached
merged_beach = pd.merge(beach_full, beach_min_time, left_on='id', right_on='id', how='inner')

# remove undrogued values using time of drogue loss
drogued_beach_ = merged_beach[merged_beach['time'] <= merged_beach['min_time']]

# remove undrogue loss time from df
drogued_beach = drogued_beach_.drop(columns=['min_time'])

In [21]:
# unbeached
merged_unbeach = pd.merge(unbeach_full, unbeach_min_time, left_on='id', right_on='id', how='inner')

# remove undrogued values using time of drogue loss
drogued_unbeach_ = merged_unbeach[merged_unbeach['time'] <= merged_unbeach['min_time']]

# remove undrogue loss time from df
drogued_unbeach = drogued_unbeach_.drop(columns=['min_time'])

In [22]:
# add column to undrogued_beach 'time to beach'
# get the last time values of beached
last_points = drogued_beach.drop_duplicates(subset='id', keep='last')

time_between_register_beaching_and_actually_beaching = 0 # immediate beaching (?)

beach_time = find_time_to_beach(drogued_beach, last_points, time_between_register_beaching_and_actually_beaching)
drogued_beach['time_to_beach'] = beach_time

In [23]:
#drogued_unbeach.to_csv('drogued_unbeach.csv', index=False) # save to csv
#drogued_beach.to_csv('drogued_beach.csv', index=False) # save to csv